**Goal of this note**:

In this note: 
* We first want to be familiar with date and time index in python (pandas and numpy), so we will be able to work on it.
* Second, we decompose a time-series into different components such as trend, seasonality, etc. 
* Then, we go for ML algorithms to predict a time series. This review covers the followings; differences of time series prediction and other ML predictions, predicting univariate time series (when the TS has 1 feature, such as sales of the company, with respect to the time) with various approaches (including ARIMA, LSTM, Prophet), how to convert a time series into a normal ML, and then see how to predict a multi-variate time series (e.g. we have different investments and sales as a function of time). (SARIMAX, LSTM, normal ML techniques, adaptive method, Robyn) 


## Overview

### 1)Time Series in Python
#### 1.1) Numpy datetime
#### 1.2) Pandas datetime index
#### 1.3) Standardizing the timeindex
##### 1.3.1) Removing duplicated indeces
##### 1.3.2) Generating complete index
#### 1.4) Resampling
#### 1.5) Variable transformation
#### 1.6) Rolling average

### 2)Time Series decomposition
### 3)Time Series prediction A: Univariate
### 4)Time Series prediction A: Multivariate

### 1) Time Series in python

#### 1.1) Numpy datetime
* This part covers basic function of time-data in numpy, pandas, and matplotlib. 
* Pandas: has built-in Time Series functionality to work with dates, date ranges, and Time Series data. It is useful for analyzing groups of time series and manipulating data. As it is based on numpy objects, we start by numpy. **Numpy** has two different time-related data-types:


* datetime64: is NumPy's datetime format, where each value is a timestamp. It was created to improve on Python's datetime format, and stores timestamps as 64-bit integers. These timestamps often default to nanosecond precision (datetime64[ns]), even when working with daily or hourly data, although this can be adjusted.
* timedelta64: is NumPy's time interval format, which can be thought of as a period of time between two datetime64 values and uses the same units as datetime64. The most common unit values are: Y: year, M: month, W: week, D: day, h: hour, m: minute, s: second, ns: nanosecond (default).



In [10]:
# imports
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from IPython.display import display
import os
#os.chdir('data')
#from colorsetup import colors, palette
#sns.set_palette(palette)
# ignore warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.1f}'.format
%matplotlib inline
plotsize = (13, 5)

In [22]:
df = pd.read_excel("Sample - Superstore.xls")
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State',
       'Country/Region', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [12]:
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country/Region,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,IN-2021-27828,2021-08-27,2021-08-31,Standard Class,VT-21700,Valerie Takahito,Home Office,Quanzhou,Fujian,China,North Asia,OFF-PA-10000026,Office Supplies,Paper,"Eaton Cards & Envelopes, 8.5 x 11",197.3,4,0.0,35.4
1,2,IN-2021-27828,2021-08-27,2021-08-31,Standard Class,VT-21700,Valerie Takahito,Home Office,Quanzhou,Fujian,China,North Asia,FUR-FU-10000944,Furniture,Furnishings,"Tenex Door Stop, Black",307.2,7,0.0,21.4
2,3,IN-2021-27828,2021-08-27,2021-08-31,Standard Class,VT-21700,Valerie Takahito,Home Office,Quanzhou,Fujian,China,North Asia,TEC-CO-10002526,Technology,Copiers,"Sharp Wireless Fax, Digital","1,422.2",4,0.0,526.2
3,4,IN-2021-27828,2021-08-27,2021-08-31,Standard Class,VT-21700,Valerie Takahito,Home Office,Quanzhou,Fujian,China,North Asia,TEC-AC-10002255,Technology,Accessories,"Memorex Keyboard, Erganomic",207.5,3,0.0,78.8
4,5,IN-2022-63178,2022-06-24,2022-06-30,Standard Class,SO-20335,Sean O'Donnell,Consumer,Marikina,Metro Manila,Philippines,Southeast Asia,OFF-AP-10002882,Office Supplies,Appliances,"KitchenAid Coffee Grinder, Silver",121.1,2,0.2,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10928,10996,IN-2021-82673,2021-09-11,2021-09-15,Standard Class,EM-13810,Eleni McCrary,Corporate,Wellington,Wellington,New Zealand,Oceania,OFF-ST-10004748,Office Supplies,Storage,"Smead Lockers, Blue",396.8,2,0.0,59.5
10929,10997,IN-2021-82673,2021-09-11,2021-09-15,Standard Class,EM-13810,Eleni McCrary,Corporate,Wellington,Wellington,New Zealand,Oceania,TEC-CO-10002678,Technology,Copiers,"Brother Wireless Fax, High-Speed","1,508.0",4,0.0,437.3
10930,10998,IN-2021-82673,2021-09-11,2021-09-15,Standard Class,EM-13810,Eleni McCrary,Corporate,Wellington,Wellington,New Zealand,Oceania,OFF-LA-10000959,Office Supplies,Labels,"Harbour Creations Round Labels, Alphabetical",6.6,1,0.0,0.6
10931,10999,ID-2020-83604,2020-08-29,2020-08-29,Same Day,PO-19195,Phillina Ober,Home Office,Rockhampton,Queensland,Australia,Oceania,FUR-BO-10002866,Furniture,Bookcases,"Safco Classic Bookcase, Pine","1,582.2",6,0.4,-870.3


In [24]:
# For the sake of simplicity, we only focus on a few columns (including date ofcourse) and name it 'base'

variables = ['Order Date', 'Category', 'Sales']
group_variables = variables[:2]
outcome_variable = variables[2]
# To calculate total sales per unique combination of Order Date and Category.
base = df.groupby(group_variables)[outcome_variable].sum().reset_index()
base.head()

,Order Date,Category,Sales
0,2019-01-01,Furniture,113.7
1,2019-01-01,Office Supplies,175.6
2,2019-01-03,Furniture,439.5
3,2019-01-03,Office Supplies,790.7
4,2019-01-03,Technology,"1,174.6"


In [25]:
df[variables].head()

,Order Date,Category,Sales
0,2021-08-27,Office Supplies,197.3
1,2021-08-27,Furniture,307.2
2,2021-08-27,Technology,"1,422.2"
3,2021-08-27,Technology,207.5
4,2022-06-24,Office Supplies,121.1


* The difference between 'base' and df[variables] is that the former is ordered in date and doesn't have duplicated dates (except when the categories are different, then we may have same date with different categories).

* We can also see types of our objects too

* **Excercise**: Work with other different options of df.groupby(group_variables)[outcome_variable].sum().reset_index() and see the difference. Also make yourself familiar with all steps of this line.


In [26]:
base.dtypes

Order Date    datetime64[ns]
Category              object
Sales                float64
dtype: object

In [27]:
# We can see how datetime64 looks like (it is in nanosecond scale)

order_date = np.array(base['Order Date'])
order_date

array(['2019-01-01T00:00:00.000000000', '2019-01-01T00:00:00.000000000',
       '2019-01-03T00:00:00.000000000', ...,
       '2022-12-31T00:00:00.000000000', '2022-12-31T00:00:00.000000000',
       '2022-12-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [28]:
# However we can use daily time:
order_date_daily = np.array(order_date, dtype='datetime64[D]')
order_date_daily

array(['2019-01-01', '2019-01-01', '2019-01-03', ..., '2022-12-31',
       '2022-12-31', '2022-12-31'], dtype='datetime64[D]')

In [29]:
# Or monthly:
order_date_monthly = np.array(order_date, dtype='datetime64[M]') # Other options: Weekly [W], yearly [Y]
order_date_monthly

array(['2019-01', '2019-01', '2019-01', ..., '2022-12', '2022-12',
       '2022-12'], dtype='datetime64[M]')

In [30]:
# We can also see how many unique values it has:
len(np.unique(order_date_monthly))

48

#### 1.2) Pandas datetime index

* We build a pandas dataframe based on numpy array. An important feature of pandas dataframe is its index which we can make it to be a datetime index and use its properties:


In [31]:
# To set the 'Order Date' column as the index of the DataFrame
base.set_index('Order Date', inplace=True)
base.head()

,Category,Sales
Order Date,,
2019-01-01,Furniture,113.7
2019-01-01,Office Supplies,175.6
2019-01-03,Furniture,439.5
2019-01-03,Office Supplies,790.7
2019-01-03,Technology,"1,174.6"


**Selecting Specific date range**:

In [33]:
# Now we can easily work with datetime index, e.g. Observations in 2014
print(base['2014'].head())
print('\n')
# Or, Observations in a range of dates, subset of columns:
print(base[base['Category'] == 'Office Supplies']['2011':'2012-02'].head())

KeyError: 'Order Date'

**Date-time component**:

In [34]:
#base.set_index('Order Date', inplace=True)
print('Day:', base.index.day, '\n')
print('Week:', base.index.week, '\n')
base['DayofWeek'] = base.index.dayofweek # Day of Week: Monday=0, Sunday=6
print(base.head())
# Note: use dt method when the date variable is not part of the index: 
# df['Order Date'].dt.dayofweek.head()
del(base['DayofWeek'])

Day: Int64Index([ 1,  1,  3,  3,  3,  4,  5,  6,  7,  7,
            ...
            27, 29, 29, 29, 30, 30, 30, 31, 31, 31],
           dtype='int64', name='Order Date', length=3101) 

Week: Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
            ...
            52, 52, 52, 52, 52, 52, 52, 52, 52, 52],
           dtype='int64', name='Order Date', length=3101) 

                   Category   Sales  DayofWeek
Order Date                                    
2019-01-01        Furniture   113.7          1
2019-01-01  Office Supplies   175.6          1
2019-01-03        Furniture   439.5          3
2019-01-03  Office Supplies   790.7          3
2019-01-03       Technology 1,174.6          3


**Excercise**:

* Assume a company is selling products, they are curious to know on which days of the week (and which days of the month) people buy more products from them. So they change their advertising strategy based on that knowledge. Can you help them based on above knowledge?

#### 1.3) Standardizing the timeindex:

* Here, we want to be sure there is 
1) No duplicate index values
2) No missing index values
in our dataset.

##### 1.3.1) Removing duplicated indexes
**Pivoting data** 

This is to remove duplicated times. Duplicated times occurs when we have sold different categories on the same date. So we need to have a column of sale for each of those categories. This is what pivot does for us.

In [ ]:
base.head()

,Category,Sales
Order Date,,
2011-01-04,Office Supplies,16.4
2011-01-05,Office Supplies,288.1
2011-01-06,Office Supplies,19.5
2011-01-07,Furniture,"2,573.8"
2011-01-07,Office Supplies,685.3


In [ ]:
base.reset_index(inplace=True)
# Note if we didn't reset the index, we could use index=None below
sales_pivot = base.pivot(index='Order Date', columns='Category', values='Sales')
sales_pivot.head()

Category,Furniture,Office Supplies,Technology
Order Date,,,
2011-01-04,NaN,16.4,NaN
2011-01-05,NaN,288.1,NaN
2011-01-06,NaN,19.5,NaN
2011-01-07,"2,573.8",685.3,"1,147.9"
2011-01-08,76.7,10.4,NaN


In [ ]:
sales_pivot.columns

Index(['Furniture', 'Office Supplies', 'Technology'], dtype='object', name='Category')

In [ ]:
# Now, we need to fill the nan values
sales_pivot.fillna(0,inplace=True)
sales_pivot.head()

Category,Furniture,Office Supplies,Technology
Order Date,,,
2011-01-04,0.0,16.4,0.0
2011-01-05,0.0,288.1,0.0
2011-01-06,0.0,19.5,0.0
2011-01-07,"2,573.8",685.3,"1,147.9"
2011-01-08,76.7,10.4,0.0


**Unstacking data**
It is another method with similar result as pivot:

In [ ]:
sales_unstack = base.set_index(['Order Date', 'Category']).unstack('Category').fillna(0)
sales_unstack.head()

Sales                           
Category   Furniture Office Supplies Technology
Order Date                                     
2011-01-04       0.0            16.4        0.0
2011-01-05       0.0           288.1        0.0
2011-01-06       0.0            19.5        0.0
2011-01-07   2,573.8           685.3    1,147.9
2011-01-08      76.7            10.4        0.0

In [ ]:
sales_unstack.columns

MultiIndex([('Sales',       'Furniture'),
            ('Sales', 'Office Supplies'),
            ('Sales',      'Technology')],
           names=[None, 'Category'])

**Excercise**

The unstacked dataframe columns is a multi-index, but as it is only sales, we don't need it. How can we make it a single index?

##### 1.3.2) Generating a complete Index

So far, we learned how to remove the duplicated indeces, but how about filling the missing dates, i.e. for example the dates the company has no sale in them.

In [ ]:
print('\nUnique dates in our data: ', len(sales_pivot.index.unique()), 'Days')
our_date_range = sales_pivot.index.max() - sales_pivot.index.min()

# Calculate number of days in date range
print('Total days in our date range:', our_date_range.days, 'Days')
#date_range = pd.date_range(min(sales.index), max(sales.index))


Unique dates in our data:  1238 Days
Total days in our date range: 1457 Days


**Date_range function**


In [ ]:
new_index = pd.date_range(sales_pivot.index.min(), sales_pivot.index.max())
sales_new = sales_pivot.reindex(new_index, fill_value=0)
sales_new

Category,Furniture,Office Supplies,Technology
2011-01-04,0.0,16.4,0.0
2011-01-05,0.0,288.1,0.0
2011-01-06,0.0,19.5,0.0
2011-01-07,"2,573.8",685.3,"1,147.9"
2011-01-08,76.7,10.4,0.0
...,...,...,...
2014-12-27,0.0,814.6,0.0
2014-12-28,0.0,13.2,164.4
2014-12-29,551.3,"1,091.2",14.9
2014-12-30,"2,330.7",282.4,302.4


#### 1.4) Resampling:

* Resampling includes two parts: Up-sampling (moving to longer periods) and Down-sampling (going to shorter periods)

**Up-sampling**:

In [ ]:
# Upsampling
sales_weekly = sales_new.resample('W').sum() # Other options are 'M' (Monthly), Q (Quarterly), and 'Y' (Yearly)
print('Weekly Sales')
sales_weekly.head()

Weekly Sales


Category,Furniture,Office Supplies,Technology
2011-01-09,"2,650.5","1,019.8","1,147.9"
2011-01-16,"1,003.8","2,039.4",827.9
2011-01-23,"1,747.3",871.1,824.1
2011-01-30,550.2,680.3,343.3
2011-02-06,290.7,502.7,649.9


**Down-Sampling**

* Just as upsampling (moving to a larger period) requires an aggregation function, downsampling (moving from Annual to Monthly, for example) requires an option to fill in missing values. A common approach is the interpolate method, which allows various types of interpolation (linear, spline, etc.). Other options (ffill forward fill, bfill backward fill) are also supported.

In [ ]:
sales_annual = sales_new.resample('Y').sum() 
# Note that downsampling (from Annual to Monthly for example) produces missing values:
sales_monthly_from_annual = sales_annual.resample('M')

#sales_monthly_from_annual.interpolate(method='linear').head()
sales_monthly_from_annual.interpolate(method='spline', order=3).head()

Category,Furniture,Office Supplies,Technology
2011-12-31,"157,192.9","151,776.4","175,278.2"
2012-01-31,"157,062.6","147,084.7","168,957.8"
2012-02-29,"157,200.9","143,355.5","164,096.2"
2012-03-31,"157,611.1","140,049.0","159,969.0"
2012-04-30,"158,251.0","137,493.3","156,975.1"


#### 1.5) Variable transformation:


* We have some options in pandas to look into variables such as difference of sales with previous day, percentage change, etc.

**Excercise**: 
* Go to these functions and find their options. Think about when these variables could be important in practice.

In [ ]:
sales_monthly = sales_new.resample('M').sum() 

# Variable First Difference
print('Monthly Sales, First Difference \n', sales_monthly.diff().head())

# Variable Percent Change
print('\nMonthly Sales % Change \n', sales_monthly.pct_change().head())

# Log Sales
print('\nlog(1+Monthly Sales) \n', np.log(1 +  sales_monthly).head())

# Add % change to original data:
sales_monthly.join(sales_monthly.pct_change().add_suffix('_%_Change')).head()

Monthly Sales, First Difference 
 Category    Furniture  Office Supplies  Technology
2011-01-31        NaN              NaN         NaN
2011-02-28   -3,821.5         -3,779.4    -1,534.8
2011-03-31   12,443.6          7,534.2    30,902.7
2011-04-30   -6,629.1          2,549.2   -23,315.7
2011-05-31   -1,032.0         -4,019.5       404.4

Monthly Sales % Change 
 Category    Furniture  Office Supplies  Technology
2011-01-31        NaN              NaN         NaN
2011-02-28       -0.6             -0.8        -0.5
2011-03-31        5.8              7.0        19.2
2011-04-30       -0.5              0.3        -0.7
2011-05-31       -0.1             -0.4         0.0

log(1+Monthly Sales) 
 Category    Furniture  Office Supplies  Technology
2011-01-31        8.7              8.5         8.1
2011-02-28        7.7              7.0         7.4
2011-03-31        9.6              9.1        10.4
2011-04-30        9.0              9.3         9.1
2011-05-31        8.8              8.9         9.

Category,Furniture,Office Supplies,Technology,Furniture_%_Change,Office Supplies_%_Change,Technology_%_Change
2011-01-31,"5,951.9","4,851.1","3,143.3",NaN,NaN,NaN
2011-02-28,"2,130.3","1,071.7","1,608.5",-0.6,-0.8,-0.5
2011-03-31,"14,574.0","8,605.9","32,511.2",5.8,7.0,19.2
2011-04-30,"7,944.8","11,155.1","9,195.4",-0.5,0.3,-0.7
2011-05-31,"6,912.8","7,135.6","9,599.9",-0.1,-0.4,0.0


#### 1.6) Rolling average:

* As a time-series can be sensitive to some noises, we can look into aggregated data (e.g. weekly data). But this would decrease the amount of data we have. Another way is to use the moving average. It kills the noises, and at the same time does not change the amount of data we have.

In [ ]:
window_size = 7
rolling_window = sales_new.rolling(window_size)
print('Rolling Mean')
print(rolling_window.mean().dropna().head())
print('\nRolling St. Dev')
print(rolling_window.std().dropna().head())
print('\nCumulative Sales')
print(sales_new.cumsum().dropna().head())


Rolling Mean
Category    Furniture  Office Supplies  Technology
2011-01-10      378.6            147.0       168.4
2011-01-11      386.1            145.1       168.4
2011-01-12      387.5            103.9       168.4
2011-01-13      387.5            101.1       168.4
2011-01-14      145.5            292.8        96.8

Rolling St. Dev
Category    Furniture  Office Supplies  Technology
2011-01-10      968.4            258.9       432.1
2011-01-11      965.2            260.1       432.1
2011-01-12      964.6            256.5       432.1
2011-01-13      964.6            257.6       432.1
2011-01-14      325.3            764.8       242.8

Cumulative Sales
Category    Furniture  Office Supplies  Technology
2011-01-04        0.0             16.4         0.0
2011-01-05        0.0            304.5         0.0
2011-01-06        0.0            324.0         0.0
2011-01-07    2,573.8          1,009.4     1,147.9
2011-01-08    2,650.5          1,019.8     1,147.9


**Excercise**:
* Plot the sales and its moved-average (with windows=1,3,7,10) and see how it is smoothed.
* How would be the weekday effect in moved average with window=7?
* See the rolling documentary for different options of rolling function.